In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.cluster import hierarchy as sch
np.set_printoptions(suppress=True)

In [18]:
df = pd.read_csv('Case3HistoricalPrices.csv')
#df1 = pd.read_csv('trt.csv')
#these are weights

In [19]:

def getIVP(cov,**kargs):
    # Compute the inverse-variance portfolio
    ivp=1./np.diag(cov)
    ivp/=ivp.sum()
    return ivp

def getClusterVar(cov,cItems):
    # Compute variance per cluster
    cov_=cov.loc[cItems,cItems] # matrix slice
    w_=getIVP(cov_).reshape(-1,1)
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0]
    return cVar

def getQuasiDiag(link):
    # Sort clustered items by distance
    link=link.astype(int)
    sortIx=pd.Series([link[-1,0],link[-1,1]])
    numItems=link[-1,3] # number of original items
    while sortIx.max()>=numItems:
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems
        sortIx[i]=link[j,0] # item 1
        df0=pd.Series(link[j,1],index=i+1)
        sortIx=sortIx.append(df0) # item 2
        sortIx=sortIx.sort_index() # re-sort
        sortIx.index=range(sortIx.shape[0]) # re-index
    return sortIx.tolist()

def getRecBipart(cov,sortIx):
    # Compute HRP alloc
    w=pd.Series(1,index=sortIx)
    cItems=[sortIx] # initialize all items in one cluster
    while len(cItems)>0:
        cItems=[i[j:k] for i in cItems for j,k in ((0,len(i)//2), (len(i)//2,len(i))) if len(i)>1] # bi-section
        for i in range(0,len(cItems),2): # parse in pairs
            cItems0=cItems[i] # cluster 1
            cItems1=cItems[i+1] # cluster 2
            cVar0=getClusterVar(cov,cItems0)
            cVar1=getClusterVar(cov,cItems1)
            alpha=1-cVar0/(cVar0+cVar1)
            w[cItems0]*=alpha # weight 1
            w[cItems1]*=1-alpha # weight 2
    return w

def correlDist(corr):
    # A distance matrix based on correlation, where 0<=d[i,j]<=1
    # This is a proper distance metric
    dist=((1-corr)/2.)**.5 # distance matrix
    return dist

In [5]:
counter = -1
weights = []

In [6]:
def allocate_portfolio(asset_prices):
    #asset_prices = asset_prices.drop(index = index)
    global counter
    global weights
    counter += 1
    df.loc[len(df)] = asset_prices
    cov,corr=df.cov(),df.corr()
    dist=correlDist(corr)
    link=sch.linkage(dist,'single')
    sortIx=getQuasiDiag(link)
    sortIx=corr.index[sortIx].tolist() # recover labels
    df0=corr.loc[sortIx,sortIx]
    hrp=np.array(getRecBipart(cov,sortIx))
    
    if(counter%63==0):
        weights = hrp
    return weights

In [7]:
allocate_portfolio(np.array(df1.loc[0]))

C:\Users\shakh\anaconda3\lib\site-packages\ipykernel_launcher.py:9: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  if __name__ == '__main__':


array([0.12315032, 0.00546421, 0.00651947, 0.01704242, 0.01523349,
       0.0172188 , 0.00978513, 0.01027385, 0.00437006, 0.01565367,
       0.00211509, 0.00202549, 0.00470787, 0.00346642, 0.00090657,
       0.00128729, 0.00073876, 0.00204557, 0.00176285, 0.00082157,
       0.00093609, 0.00128422, 0.68359073, 0.06960007])